# Stage 1: Build a database

## Basic match dataframe setup

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("football_database.sqlite")
df = pd.read_sql_query("""
        SELECT id, country_id, league_id, season, stage, date, 
            match_api_id, home_team_api_id, away_team_api_id, 
            home_team_goal, away_team_goal, home_player_1, home_player_2, 
            home_player_3, home_player_4, home_player_5, 
            home_player_6, home_player_7, home_player_8, 
            home_player_9, home_player_10, home_player_11, 
            away_player_1, away_player_2, away_player_3, 
            away_player_4, away_player_5, away_player_6, 
            away_player_7, away_player_8, away_player_9, 
            away_player_10, away_player_11, goal, shoton, 
            shotoff, foulcommit, card, cross, corner, possession
        FROM match
        """, conn)


In [2]:
# drop null values
df.dropna(inplace=True)

df.shape

(13325, 41)

In [3]:
# Change player ids to int instead of float

home_fields = [f'home_player_{i}' for i in range(1, 12)]
away_fields = [f'away_player_{i}' for i in range(1, 12)]

df[home_fields] = df[home_fields].astype(int)
df[away_fields] = df[away_fields].astype(int)

df['date'] = df['date'].apply(pd.to_datetime)

In [4]:
# Make df smaller for faster processing
df = df.head(5)

In [5]:
# Add match_result

df.loc[df['home_team_goal'] > df['away_team_goal'], 'match_result'] = 1
df.loc[df['home_team_goal'] < df['away_team_goal'], 'match_result'] = 2
df.loc[df['home_team_goal'] == df['away_team_goal'], 'match_result'] = 3
df['match_result'] = df['match_result'].astype(int)

# Get the first year of season
df['season'] = df['season'].apply(lambda x: x[:4])

df.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,match_result
1728,1729,1729,1729,2008,1,2008-08-17,489042,10260,10261,1,...,37799,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...,3
1729,1730,1729,1729,2008,1,2008-08-16,489043,9825,8659,1,...,27267,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...,1
1730,1731,1729,1729,2008,1,2008-08-16,489044,8472,8650,0,...,30853,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2
1731,1732,1729,1729,2008,1,2008-08-16,489045,8654,8528,2,...,34466,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...,1
1733,1734,1729,1729,2008,1,2008-08-16,489047,8668,8655,2,...,30646,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><event...,2


In [6]:
## Calculate lineup average age

# Load birth year data
birth_day = pd.read_sql_query("""
        SELECT player_api_id as id, birthday
        FROM player
""", conn, params=())

birth_day.head(5)

,id,birthday
0,505942,1992-02-29 00:00:00
1,155782,1989-12-15 00:00:00
2,162549,1991-05-13 00:00:00
3,30572,1982-05-08 00:00:00
4,23780,1979-11-08 00:00:00


In [7]:
# Get average age of a line up for home players

dates = df[home_fields].astype(int).replace(to_replace=birth_day['id'].tolist(), value=birth_day['birthday'].tolist())
dates['date'] = df['date']

dates.head(5)

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,date
1728,1970-10-29 00:00:00,1979-10-13 00:00:00,1978-11-07 00:00:00,1981-10-21 00:00:00,1981-05-15 00:00:00,1984-02-01 00:00:00,1981-07-28 00:00:00,1974-11-16 00:00:00,1973-11-29 00:00:00,1987-09-13 00:00:00,1985-10-24 00:00:00,2008-08-17
1729,1977-05-19 00:00:00,1983-02-14 00:00:00,1977-08-17 00:00:00,1987-01-18 00:00:00,1985-07-26 00:00:00,1989-03-16 00:00:00,1983-06-04 00:00:00,1988-02-16 00:00:00,1987-06-26 00:00:00,1984-02-26 00:00:00,1988-01-16 00:00:00,2008-08-16
1730,1982-12-31 00:00:00,1979-02-21 00:00:00,1980-10-11 00:00:00,1980-08-06 00:00:00,1985-06-28 00:00:00,1980-01-06 00:00:00,1979-11-27 00:00:00,1982-07-29 00:00:00,1984-10-21 00:00:00,1981-01-15 00:00:00,1983-03-15 00:00:00,2008-08-16
1731,1980-01-18 00:00:00,1985-04-19 00:00:00,1979-04-18 00:00:00,1983-01-01 00:00:00,1978-03-09 00:00:00,1983-08-01 00:00:00,1987-05-08 00:00:00,1980-10-13 00:00:00,1981-08-14 00:00:00,1983-11-12 00:00:00,1983-11-24 00:00:00,2008-08-16
1733,1979-03-06 00:00:00,1977-01-21 00:00:00,1980-09-06 00:00:00,1982-08-16 00:00:00,1974-09-12 00:00:00,1982-03-26 00:00:00,1991-03-11 00:00:00,1982-08-17 00:00:00,1984-12-11 00:00:00,1981-05-17 00:00:00,1982-11-22 00:00:00,2008-08-16


In [8]:
# Calculate average difference from player ages
dates = dates.apply(pd.to_datetime)

dates['avg_home_ages'] = dates.apply(lambda s: dates['date']-s).astype('timedelta64[Y]').mean(axis=1).astype(int)

df = df.join(dates['avg_home_ages']).drop('id', axis=1)

df.head(5)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,match_result,avg_home_ages
1728,1729,1729,2008,1,2008-08-17,489042,10260,10261,1,1,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...,3,25
1729,1729,1729,2008,1,2008-08-16,489043,9825,8659,1,0,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...,1,21
1730,1729,1729,2008,1,2008-08-16,489044,8472,8650,0,1,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2,24
1731,1729,1729,2008,1,2008-08-16,489045,8654,8528,2,1,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...,1,23
1733,1729,1729,2008,1,2008-08-16,489047,8668,8655,2,3,...,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><event...,2,24


In [9]:
# Get average age of a line up for away players

dates = df[away_fields].astype(int).replace(to_replace=birth_day['id'].tolist(), value=birth_day['birthday'].tolist())
dates['date'] = df['date']

dates.head(5)

,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,date
1728,1976-04-20 00:00:00,1977-10-19 00:00:00,1986-01-23 00:00:00,1982-01-22 00:00:00,1986-05-28 00:00:00,1986-01-04 00:00:00,1987-04-18 00:00:00,1975-01-21 00:00:00,1983-07-05 00:00:00,1979-03-02 00:00:00,1984-10-28 00:00:00,2008-08-17
1729,1985-09-03 00:00:00,1978-10-06 00:00:00,1985-11-30 00:00:00,1980-10-06 00:00:00,1978-12-14 00:00:00,1979-01-02 00:00:00,1982-07-14 00:00:00,1984-12-14 00:00:00,1986-05-25 00:00:00,1983-01-26 00:00:00,1987-03-05 00:00:00,2008-08-16
1730,1982-08-31 00:00:00,1983-01-17 00:00:00,1978-01-28 00:00:00,1973-10-07 00:00:00,1981-09-11 00:00:00,1980-07-22 00:00:00,1980-05-30 00:00:00,1988-03-05 00:00:00,1980-05-05 00:00:00,1980-07-08 00:00:00,1984-03-20 00:00:00,2008-08-16
1731,1981-05-02 00:00:00,1976-11-04 00:00:00,1979-09-24 00:00:00,1983-05-02 00:00:00,1980-03-11 00:00:00,1985-08-04 00:00:00,1988-03-21 00:00:00,1984-07-29 00:00:00,1980-09-27 00:00:00,1983-04-01 00:00:00,1978-01-11 00:00:00,2008-08-16
1733,1979-10-15 00:00:00,1974-07-11 00:00:00,1984-03-28 00:00:00,1977-10-18 00:00:00,1981-12-12 00:00:00,1979-12-27 00:00:00,1980-11-25 00:00:00,1981-09-08 00:00:00,1981-03-10 00:00:00,1978-01-25 00:00:00,1981-08-16 00:00:00,2008-08-16


In [10]:
# Calculate average difference from player ages
dates = dates.apply(pd.to_datetime)

dates['avg_away_ages'] = dates.apply(lambda s: dates['date']-s).astype('timedelta64[Y]').mean(axis=1).astype(int)

df = df.join(dates['avg_away_ages'])

df.head(5)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,shoton,shotoff,foulcommit,card,cross,corner,possession,match_result,avg_home_ages,avg_away_ages
1728,1729,1729,2008,1,2008-08-17,489042,10260,10261,1,1,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...,3,25,23
1729,1729,1729,2008,1,2008-08-16,489043,9825,8659,1,0,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...,1,21,22
1730,1729,1729,2008,1,2008-08-16,489044,8472,8650,0,1,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2,24,24
1731,1729,1729,2008,1,2008-08-16,489045,8654,8528,2,1,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...,1,23,24
1733,1729,1729,2008,1,2008-08-16,489047,8668,8655,2,3,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><event...,2,24,25


## Task 1A: Parse XML fields

In [11]:
# Create a helper function that counts team nodes
from collections import Counter
import xml.etree.ElementTree as ET

def count_teams(xml_string):
    c = Counter()
    root = ET.fromstring(xml_string)
    for t in root.iter('team'):
        c.update([t.text])
    return c

In [12]:
# Add extracted XML info to the dataframe

df['home_shoton'] = df.apply(lambda s: count_teams(s['shoton'])[str(s['home_team_api_id'])], axis=1)
df['away_shoton'] = df.apply(lambda s: count_teams(s['shoton'])[str(s['away_team_api_id'])], axis=1)

df['home_shotoff'] = df.apply(lambda s: count_teams(s['shotoff'])[str(s['home_team_api_id'])], axis=1)
df['away_shotoff'] = df.apply(lambda s: count_teams(s['shotoff'])[str(s['away_team_api_id'])], axis=1)

df['home_foulcommit'] = df.apply(lambda s: count_teams(s['foulcommit'])[str(s['home_team_api_id'])], axis=1)
df['away_foulcommit'] = df.apply(lambda s: count_teams(s['foulcommit'])[str(s['away_team_api_id'])], axis=1)

df['home_card'] = df.apply(lambda s: count_teams(s['card'])[str(s['home_team_api_id'])], axis=1)
df['away_card'] = df.apply(lambda s: count_teams(s['card'])[str(s['away_team_api_id'])], axis=1)

df['home_corner'] = df.apply(lambda s: count_teams(s['corner'])[str(s['home_team_api_id'])], axis=1)
df['away_corner'] = df.apply(lambda s: count_teams(s['corner'])[str(s['away_team_api_id'])], axis=1)

df['away_pos'] = df['possession'].apply(lambda s: ET.fromstring(s).findall('./value/awaypos')[-1].text)
df['home_pos'] = df['possession'].apply(lambda s: ET.fromstring(s).findall('./value/homepos')[-1].text)

df.head(5)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,home_shotoff,away_shotoff,home_foulcommit,away_foulcommit,home_card,away_card,home_corner,away_corner,away_pos,home_pos
1728,1729,1729,2008,1,2008-08-17,489042,10260,10261,1,1,...,10,9,16,11,3,0,6,6,45,55
1729,1729,1729,2008,1,2008-08-16,489043,9825,8659,1,0,...,13,3,11,9,0,0,7,5,34,66
1730,1729,1729,2008,1,2008-08-16,489044,8472,8650,0,1,...,3,5,13,12,0,2,1,8,54,46
1731,1729,1729,2008,1,2008-08-16,489045,8654,8528,2,1,...,7,15,14,13,2,1,6,10,48,52
1733,1729,1729,2008,1,2008-08-16,489047,8668,8655,2,3,...,7,8,11,11,2,2,3,4,49,51


## Task 1B: Parsing player attributes

In [13]:
# Load Player attributes
player_attrib = pd.read_sql_query("""
        SELECT  id, player_api_id as player_id, date, overall_rating, crossing, finishing, heading_accuracy,
                short_passing, volleys, dribbling, curve, free_kick_accuracy,
                long_passing, ball_control, acceleration, sprint_speed, agility,
                reactions, balance, shot_power, jumping, stamina, strength,
                long_shots, aggression, interceptions, positioning, vision,
                penalties, marking, standing_tackle, sliding_tackle
        FROM player_attributes
""", conn, params=())

player_attrib['date'] = player_attrib['date'].apply(pd.to_datetime)

player_attrib.head(5)

,id,player_id,date,overall_rating,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
0,1,505942,2016-02-18,67.0,49.0,44.0,71.0,61.0,44.0,51.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
1,2,505942,2015-11-19,67.0,49.0,44.0,71.0,61.0,44.0,51.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
2,3,505942,2015-09-21,62.0,49.0,44.0,71.0,61.0,44.0,51.0,...,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0
3,4,505942,2015-03-20,61.0,48.0,43.0,70.0,60.0,43.0,50.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0
4,5,505942,2007-02-22,61.0,48.0,43.0,70.0,60.0,43.0,50.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0


In [14]:
# Get the attributes for each player closest to match point
target = df[['date'] + home_fields].copy()

def get_most_recent_attributes(player, field):
    earlier_dates = player_attrib.loc[(player_attrib['player_id'] == player[field]) 
                                      & (player_attrib['date'] < pd.to_datetime(player['date']))]
    return earlier_dates[earlier_dates['date'] == earlier_dates['date'].max()].index[0]

# Map the ids to the corresponding attribute ids
for col in home_fields:
    target[col] = target.apply(lambda s: get_most_recent_attributes(s, col), axis=1)

target.head(5)

,date,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11
1728,2008-08-17,48276,179124,149861,131150,139238,38352,122758,140510,155486,57592,178974
1729,2008-08-16,108719,18518,179774,84075,60709,169567,50140,42388,156892,50031,131822
1730,2008-08-16,32405,139070,134972,37260,143004,164746,169311,13139,96785,48526,38671
1731,2008-08-16,150378,176073,119962,25074,104657,91768,114713,158629,119660,26441,41653
1733,2008-08-16,171759,143112,89388,85064,134889,126085,74982,143029,100505,100989,3977


In [ ]:
ratings = target[home_fields].replace(to_replace=player_attrib.index.tolist(), value=player_attrib['overall_rating'].tolist())

ratings.head()

In [ ]:
ratings['home_overall_ratings'] = ratings.mean(axis=1).astype(int)

df.join(ratings['home_overall_ratings'])